<a href="https://colab.research.google.com/github/Itallo0708/Computational-Mathematics/blob/main/notebooks/atividade_02_no_animations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Questão A)
Inicialmente importando bibliotecas e definindo funções essenciais para a execução do algoritmo do gradiente descendente.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML

def calculate_ssr(intercept, slope, x, y):
    predictions = intercept + (slope * x)
    residuals = y - predictions
    ssr = np.sum(residuals ** 2)
    return ssr

def gradient_descent(learning_rate, start_intercept, x, y, slope, max_iterations=40, tolerance=0.001):
    # Executa o algoritmo de Gradiente Descendente.

    current_intercept = start_intercept
    history = []

    print(f"\n INICIANDO COM LEARNING RATE = {learning_rate}")

    # Adiciona o estado inicial.
    current_ssr = calculate_ssr(current_intercept, slope, x, y)

    # Calcula gradiente inicial.
    predictions = current_intercept + (slope * x)
    residuals = y - predictions
    gradient = -2 * np.sum(residuals)

    history.append((current_intercept, current_ssr, gradient))

    for i in range(max_iterations):
        # Calcular Gradiente.
        predictions = current_intercept + (slope * x)
        residuals = y - predictions
        gradient = -2 * np.sum(residuals)

        # Calcular Passo.
        step_size = gradient * learning_rate

        # Intercept.
        old_intercept = current_intercept
        new_intercept = old_intercept - step_size
        current_intercept = new_intercept

        # Novo Erro.
        current_ssr = calculate_ssr(current_intercept, slope, x, y)

        # Salvar Histórico.
        history.append((current_intercept, current_ssr, gradient))

        print(f"Iteração {i+1}: Old={old_intercept:.4f} | Grad={gradient:.4f} | Step={step_size:.4f} | New={new_intercept:.4f}")

        if abs(step_size) < tolerance:
            print(f"--> Convergiu na iteração {i+1}!")
            break

    return history

def setup_regression_view(ax, x_data, y_data, learning_rate):
    # gráfico da Reta
    ax.set_xlim(0, 3.5)
    ax.set_ylim(0, 4)

    # Plotar dados
    ax.scatter(x_data, y_data, color='green', s=100, label='Dados Reais', zorder=5)

    # objeto da reta vazio inicialmente
    line, = ax.plot([], [], 'b-', linewidth=3, label='Reta Prevista')

    ax.set_title(f'Ajuste da Reta (LR={learning_rate})')
    ax.set_xlabel('Peso (x)')
    ax.set_ylabel('Altura (y)')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)

    return line

def setup_cost_view(ax, cost_function, x_data, y_data, fixed_slope):
    # gráfico Curva de Custo
    # Gerar dados para a curva
    curve_x = np.linspace(-0.5, 2.5, 100)
    curve_y = [cost_function(b, fixed_slope, x_data, y_data) for b in curve_x]

    # Plotar a curva
    ax.plot(curve_x, curve_y, 'teal', alpha=0.5, linewidth=2, label='Curva de Custo (SSR)')

    # Configurar limites
    ax.set_xlim(-0.5, 2.5)
    ax.set_ylim(0, max(curve_y) + 1)

    # Criar elementos móveis (inicialmente vazios)
    dot, = ax.plot([], [], 'ro', markersize=10, zorder=5, label='Valor Atual')
    track, = ax.plot([], [], 'r--', alpha=0.3) # Rastro
    tangent_line, = ax.plot([], [], 'orange', linewidth=2, label='Derivada')

    # Caixa de texto informativa
    info_text = ax.text(0.5, 0.85, '', transform=ax.transAxes, ha='right',
                        bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))

    ax.set_title('Descida do Gradiente')
    ax.set_xlabel('Intercepto')
    ax.set_ylabel('Erro (SSR)')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

    return dot, track, tangent_line, info_text

def create_animation(history, x_data, y_data, fixed_slope, learning_rate):
    # montar a figura e a animação

    # Criar Figura e Eixos
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    plt.subplots_adjust(wspace=0.3)

    line_reg = setup_regression_view(ax1, x_data, y_data, learning_rate)

    dot, track, tangent, info_txt = setup_cost_view(ax2, calculate_ssr, x_data, y_data, fixed_slope)

    def update(frame):
        # Pegando dados
        b, ssr, grad = history[frame]

        # Atualizar reta
        x_vals = np.array([0, 3.5])
        y_vals = b + (fixed_slope * x_vals)
        line_reg.set_data(x_vals, y_vals)

        # Atualizar curva
        history_x = [h[0] for h in history[:frame+1]]
        history_y = [h[1] for h in history[:frame+1]]

        dot.set_data([b], [ssr])
        track.set_data(history_x, history_y)

        # Atualizar Tangente
        tan_range = 0.6
        x_tan = np.linspace(b - tan_range/2, b + tan_range/2, 10)
        y_tan = grad * (x_tan - b) + ssr
        tangent.set_data(x_tan, y_tan)

        # Atualizar Texto
        info_txt.set_text(
            f'Iteração: {frame}\nIntercepto: {b:.2f}\nErro: {ssr:.2f}\nGrad: {grad:.2f}'
        )

        return line_reg, dot, track, tangent, info_txt
    ani = FuncAnimation(fig, update, frames=len(history), blit=True, interval=150)
    return ani






# Gradiente Descendente
O objetivo desse código foi fazer uso do algoritmo de Gradiente Descendente para ajustar um modelo de Regressão Linear Simples, o qual tem a finalidade de encontrar o melhor valor do intercept (*b*) , nesse caso, mantendo constante o coeficiente angular (*m*).

Nesse cenário, temos que a reta da regressão linear, técnica a qual visa traçar uma reta que represente de forma eficiente o conjunto de dados, se dá por:
$$
y_{\text{pred}} = m \cdot x + b
$$
Nesse caso iremos fixar o *m* = 0.64 e iremos usar o gradiente descendente para otimizar o valor de *b*.
# Função de custo (Loss Function)
Para o processo de otimização, é essencial calcular o erro do modelo de previsão, isto é, o quão distante a reta estimada está dos pontos reais. Nesse contexto, a função de custo representa matematicamente essa distância entre os valores previstos pelo modelo e os valores observados nos dados.

Para isso, utilizamos a Soma dos Resíduos ao Quadrado (*SSR*), que se dá por:
$$
f(b) = \sum_{i=1}^{n} \left( y_{\text{real}}^{(i)} - (m \cdot x^{(i)} + b) \right)^2
$$
Visualmente, esta função forma uma parábola convexa, onde o ponto mínimo representa o intercepto ideal.
# Calculo do Gradiente (Derivada)
Para minimizar $f(b)$, calculamos a derivada da função de custo em relação ao intercepto $b$. Pela regra da cadeia, a derivada é:
$$\frac{df}{db} = -2 \sum_{i=1}^{n} (y_{real}^{(i)} - y_{pred}^{(i)})$$
Este valor nos indica a inclinação da curva de custo e a direção para onde devemos ajustar $b$
# Step Size
Uma vez calculada a derivada (gradiente), sabemos a direção para onde o erro diminui, mas ainda precisamos definir o tamanho do ajuste a ser feito no intercepto. É aqui que entra a Taxa de Aprendizado.

O tamanho real do passo (Step Size) que o algoritmo dá em cada iteração é o produto da derivada pela taxa de aprendizado:
$$\text{Step Size} = \text{Gradiente} \times \ learning \ rate$$
Este mecanismo cria um comportamento autoajustável, o qual, quando o erro é alto, a curva é íngreme (gradiente alto), gerando passos grandes para avançar rápido.

Por outro lado, conforme nos aproximamos do fundo da parábola, a inclinação diminui (gradiente baixo), gerando passos cada vez menores ("baby steps") para evitar ultrapassar o ponto ótimo e garantir uma convergência precisa.

Dessa forma, o novo valor do intercept $b$ será de:
$$b_{\text{novo}} = b_{\text{atual}} - \underbrace{(\text{Gradiente} \times \ lerning \ rate)}_{\text{Step Size}}$$

# Convergência e critério de parada
O processo de otimização descrito acima é iterativo. O algoritmo repete os passos de cálculo do gradiente e atualização do intercepto sucessivas vezes.

inicialmente, por estarmos longe do ideal, a inclinação da curva é acentuada (gradiente alto em magnitude), o que resulta em passos largos ($StepSize$ grande) para avançar rapidamente.

No processo de aproximação, conforme o intercept desliza em direção ao fundo da parábola, a curva se torna menos íngreme. O valor do gradiente diminui naturalmente.

No ponto exato onde o erro é mínimo, a reta tangente é perfeitamente horizontal. Matematicamente, isso significa que a derivada é zero ($\frac{df}{db} = 0$).

Na prática, é  raro atingirmos o zero absoluto devido à precisão de ponto flutuante. Portanto, definimos um critério de convergência no qual o algoritmo para quando o tamanho do passo ($StepSize$) se torna menor que uma tolerância pré-definida.

# Exemplo 1: learning hate = 0.01
Nesse cenário, usaremos os pontos:

Ponto 1: $x=0.5, y=1.4$

Ponto 2: $x=2.3, y=1.9$

Ponto 3: $x=2.9, y=3.2$

assumimos que a inclinação da reta (o coeficiente angular $m$) é conhecida e fixa em 0.64. Nosso objetivo é descobrir onde a reta deve cruzar o eixo vertical (o intercept $b$).

Começando com um chute arbitrário de que o intercepto é 0. A reta prevista é $$y = 0.64x + 0$$
Aplicando os valores previstos na reta na fórmula do custo temos:
$$f(b) = \sum_{i=1}^{n} \left( y_{\text{real}}^{(i)} - (0.64 \cdot x^{(i)} + 0) \right)^2 = 3.15$$
Concluindo que a reta passa distante dos pontos.

Aplicando o cálculo do gradiente temos:
$$\frac{df}{db} = -2 \sum_{i=1}^{n} (y_{real}^{(i)} - y_{pred}^{(i)}) = -5.7$$
Assim, tem-se uma forte inclinação descendente indicando que o ponto está à esquerda do ponto ótimo.

Em seguida, o algoritmo usa o gradiente para calcular o próximo passo:
$$\text{Step Size} = \text{Gradiente} (-5.7) \times \text{Taxa de Aprendizado} (0.01) = -0.057$$
E com isso ajusta o valor do novo intercept:
$$b_{new} = b_{0} - \text{(-0.057)}$$

Desse modo, o processo se repete até que o gradiente seja praticamente nulo, a reta tangente no gráfico da direita fica horizontal. No nosso exemplo, isso ocorre próximo ao valor de 0.87. Neste ponto, atingimos o Mínimo Global da função de custo para este conjunto de dados. A reta $y = 0.64x + 0.87$ é a melhor aproximação linear possível para representar a relação entre Peso e Altura nestas observações.


In [ ]:
# Dados.
weights = np.array([0.5, 2.3, 2.9])
heights_real = np.array([1.4, 1.9, 3.2])
fixed_slope = 0.64

history_data = gradient_descent(0.01, 0, weights, heights_real, fixed_slope)
animacao = create_animation(history_data, weights, heights_real, fixed_slope, 0.01)

HTML(animacao.to_jshtml())

# Exemplo 2: Learning rate = 0.05

No exemplo a seguir foram mantidos os mesmos dados com a exceção do learning rate que inicialmente era *0.01* e agora será de *0.05*.

Assim, ao executar o mesmo algoritmo, mas agora em um cenário com um learning hate consideravelmente maior, temos que o gradiente converge a 0 muito mais rapidamente, pois com um learning rate maior o algoritmo consegue chegar em um valor melhor de intercept em menos interações.


In [ ]:
history_data = gradient_descent(0.05, 0, weights, heights_real, fixed_slope)
animacao = create_animation(history_data, weights, heights_real, fixed_slope, 0.05)

HTML(animacao.to_jshtml())

# Questão B)
Adaptando a função usada anteriormente para executar o algoritmo com intercept e slope variando.

In [2]:
def gradient_descent_two_parameters(learning_rate, start_intercept, x, y, start_slope, max_iterations=40, tolerance=0.001):
    # Executa o algoritmo de Gradiente Descendente.

    current_intercept = start_intercept
    current_slope = start_slope
    history = []

    print(f"\n INICIANDO COM LEARNING RATE = {learning_rate}")

    # Adiciona o estado inicial.
    current_ssr = calculate_ssr(current_intercept, current_slope, x, y)

    # Calcula gradiente inicial.
    predictions = current_intercept + (current_slope * x)
    residuals = y - predictions
    d_intercept = -2 * np.sum(residuals)
    d_slope = -2 * np.sum(residuals * x)

    history.append((current_intercept, current_ssr, d_intercept, d_slope))

    for i in range(max_iterations):
        # Calcular Gradiente.
        predictions = current_intercept + (current_slope * x)
        residuals = y - predictions
        d_intercept = -2 * np.sum(residuals)
        d_slope = -2 * np.sum(residuals * x)

        # Calcular Passo.
        step_size_intercept = d_intercept * learning_rate
        step_size_slope = d_slope * learning_rate

        # Intercept.
        old_intercept = current_intercept
        new_intercept = old_intercept - step_size_intercept
        current_intercept = new_intercept

        # slope.
        old_slope = current_slope
        new_slope = old_slope - step_size_slope
        current_slope = new_slope

        # Novo Erro.
        current_ssr = calculate_ssr(current_intercept, current_slope, x, y)

        # Salvar Histórico.
        history.append((current_intercept, current_ssr, d_intercept, d_slope))

        print(f"Iteração {i+1}: Old i={old_intercept:.4f} | old s={old_slope:.4f} | d/di={d_intercept:.4f} |d/ds={d_slope:.4f} | Step i={step_size_intercept:.4f} |step s={step_size_slope:.4f} | New i={new_intercept:.4f} | New s={new_slope:.4f}")

        if abs(step_size_intercept) < tolerance and abs(step_size_slope) < tolerance:
            print(f"--> Convergiu na iteração {i+1}!")
            break

    return history

def create_line_animation_two_parameters(history, x_data, y_data, learning_rate, title=""):
    # 1. Criar Figura
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_xlim(0, 3.5)
    ax.set_ylim(0, 4)

    # Plotar os dados
    ax.scatter(x_data, y_data, color='green', s=100, label='Dados Reais', zorder=5)

    # Criar a linha da reta
    line_reg, = ax.plot([], [], 'b-', linewidth=3, label='Reta Prevista')

    # Caixa de texto para mostrar os valores
    info_text = ax.text(0.05, 0.8, '', transform=ax.transAxes, ha='left',
                        bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))

    ax.set_title(f'{title} com (LR={learning_rate})')
    ax.set_xlabel('Peso (x)')
    ax.set_ylabel('Altura (y)')
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)

    # Função de Atualização
    def update(frame):
        b, m, gb, gm = history[frame]
        x_vals = np.array([0, 3.5])
        y_vals = b + (m * x_vals)

        # Atualizar o gráfico
        line_reg.set_data(x_vals, y_vals)

        # Atualizar o texto com os valores
        info_text.set_text(
            f'Iteração: {frame}\n'
            f'b (Intercept): {b:.2f}\n'
            f'm (Slope): {m:.2f}\n'
            f'gradient intercept : {gb:.2f}\n'
            f'gradient slope: {gm:.2f}'
        )

        return line_reg, info_text

    # Criar Animação
    ani = FuncAnimation(fig, update, frames=len(history), blit=True, interval=100)

    return ani

# Exemplo 1: Learning Rate = 0.01
Nesse exemplo iremos ultilizar os mesmos valores da questão anterior, com o diferencial de que o slope não será fixo.

Inicialmente escolheremos o $intercept$ como $b = 0$, e o $slope$ como $m = 1$.

A reta prevista é $$y = 1x + 0$$

E a Soma dos Erros ao Quadrado será $$f(b) = \sum_{i=1}^{n} \left( y_{\text{real}}^{(i)} - (1 \cdot x^{(i)} + 0) \right)^2 = 1.06$$
Concluindo que a reta passa distante dos pontos.

Nesse contexto, com 2 parâmetros, a principal mudança é que faremos uma derivada em relação ao $intercept$:
$$
\frac{\partial \text{SSR}}{\partial b}
=
-2 \sum_{i=1}^{n} \left( y_i - (b + m x_i) \right)
$$

E outra em relação ao $slope$:
$$
\frac{\partial \text{SSR}}{\partial m}
=
-2 \sum_{i=1}^{n} x_i \left( y_i - (b + m x_i) \right)
$$

Após essa etapa teremos o tamanho do passo para o novo $intercept$:
$$ step\ size\ intercept =\frac{\partial \text{SSR}}{\partial b} \ learning \ rate$$
Novo $intercept$:
$$ intercept \ novo = intercept - \ step \ size\ intercept$$

E teremos o tamanho do passo para o novo $intercept$:
$$ step\ size\ slope =\frac{\partial \text{SSR}}{\partial m} \ learning \ rate$$

Novo $slope$:
$$ slope \ novo = slope - \ step \ size\ slope$$

Dessa forma, a cada iteração do algoritmo os valores de $intercept$ e $slope$ vão sendo ajustados, e os gradientes de ambos os parâmetros devem se aproximar de 0, objetivando o menor valor possível de inclinação da reta tangente para ambos os parâmetros.

In [3]:
weights = np.array([0.5, 2.3, 2.9])
heights_real = np.array([1.4, 1.9, 3.2])

In [ ]:
teste = gradient_descent_two_parameters(0.01, 0, weights, heights_real, 1, max_iterations=40, tolerance=0.001 )
animacao_b = create_line_animation_two_parameters(teste, weights, heights_real, 0.01, "Regressão com 2 Parâmetros")
HTML(animacao_b.to_jshtml())

# Exemplo: Learning rate = 0.05
Ademais, aplicando o mesmo algoritmo a uma taxa de aprendizado maior iremos obter o seguinte resultado:

In [ ]:
exemplo4 = gradient_descent_two_parameters(0.05, 0, weights, heights_real, 1, max_iterations=40, tolerance=0.001 )
animacao_exemplo4= create_line_animation_two_parameters(teste, weights, heights_real, 0.05, "Regressão com 2 Parâmetros")
HTML(animacao_exemplo4.to_jshtml())

No experimento, o gradiente descendente que ajusta apenas o intercepto 𝑏 convergiu mais rápido do que a versão que ajusta simultaneamente 𝑏 e o slope 𝑚, pois o valor inicial do slope já estava muito próximo do ótimo ($𝑚≈0,64$). Como a inclinação já estava ajustada no início, o algoritmo precisou apenas realizar pequenos deslocamentos verticais para reduzir o erro.

Nesse caso específico, o chute inicial ter sido distante do valor ótimo acabou aumentando o número de iterações, no entanto, em termos gerais, a otimização simultânea de todos os parâmetros nos daria um erro menor, visto que a reta poderia ser ajustada tanto em altura, quanto em inclinação em cenários desfavoráveis.

# Gradiente Descendente Estocástico
Adaptando o código para um modelo estocástico temos:

In [4]:
def stochastic_gradient_descent(learning_rate, start_intercept, x, y, start_slope, max_iterations=40, tolerance=0.001):
    # Executa o algoritmo de Gradiente Descendente.

    current_intercept = start_intercept
    current_slope = start_slope
    history = []

    print(f"\n INICIANDO SGD COM LEARNING RATE = {learning_rate}")

    # Adiciona o estado inicial.
    current_ssr = calculate_ssr(current_intercept, current_slope, x, y)
    history.append((current_intercept, current_ssr, 0, 0))

    for i in range(max_iterations):
      # sorteio
      random_index = np.random.randint(0, len(x))
      x_i = x[random_index]
      y_i = y[random_index]

      # calcular predict e erro
      predictions = current_intercept + (current_slope * x_i)
      residuals = y_i - predictions

      # gradiente
      d_intercept = -2 * residuals
      d_slope = -2 * residuals * x_i

      # Calcular Passo.
      step_size_intercept = d_intercept * learning_rate
      step_size_slope = d_slope * learning_rate

      # Intercept.
      old_intercept = current_intercept
      new_intercept = old_intercept - step_size_intercept
      current_intercept = new_intercept

      # slope.
      old_slope = current_slope
      new_slope = old_slope - step_size_slope
      current_slope = new_slope

      # Novo Erro.
      current_ssr = calculate_ssr(current_intercept, current_slope, x, y)

      # Salvar Histórico.
      history.append((current_intercept, current_ssr, d_intercept, d_slope))

      print(f"Iteração {i+1}: ponto:{random_index} | Old i={old_intercept:.4f} | old s={old_slope:.4f} | d/di={d_intercept:.4f} |d/ds={d_slope:.4f} | Step i={step_size_intercept:.4f} |step s={step_size_slope:.4f} | New i={new_intercept:.4f} | New s={new_slope:.4f}")

      if abs(step_size_intercept) < tolerance and abs(step_size_slope) < tolerance:
          print(f"--> Convergiu na iteração {i+1}!")
          break

    return history

# Diferenças Para o Gradiente Descendente Tradicional
No Gradiente Descendente padrão, para dar um único passo, o algoritmo precisa:
Calcular o erro de todos os pontos de dados. Somar todos esses erros, calcular a derivada do somátorio dos erros ao quadrado, e só então atualizar o $intercept$ e o $slope$.

Nesse contexto, A ideia do Estocástico é em vez de olhar para todos os dados para dar um passo, vamos sortear apenas ponto de dado e usar ele para decidir a direção.

# Algoritmo
Começamos com valores aleatórios, por exemplo, $Intercept=0$, $Slope=1$, igual ao método tradicional. Em seguida, ao invés de calcularmos o erro em todos os pontos, escolhe-se um ponto de forma aleatória e executa-se o cálculo da derivada com base apenas nesse ponto.

Com isso temos que a derivada em relação ao $Interept$ é:
$$
Gradiente_{intercept} = -2(y_{real} - y_{pred})
$$
E a atualização do $Intercept$ se dá por:
$$
Intercept_{novo} = Intercept_{velho} - (Learning Rate \times Gradiente)
$$
Com isso temos que a derivada em relação ao $Slope$ é:
$$
Gradiente_{slope} = -2(y_{real} - y_{pred})x
$$
E a atualização do $Slope$ se dá por:
$$
Slope_{novo} = Slope_{velho} - (Learning Rate \times Gradiente)
$$

Dessa maneira, após o cálculo do novo $Intercept$ e $Slope$, outro ponto é sorteado e o processo é repetido até alcançarmos uma condição de parada, no caso, iremos definir um limite de iterações e um tamanho mínimo do passo.

# Exemplo: Gradiente Estocástico com Learning Rate = 0.05
Nesse exemplo, iniciamos com um chute inicial para o $Intercept = 0$ e $ Slope = 1$

In [ ]:
exemplo5 = stochastic_gradient_descent(0.05, 0, weights, heights_real, 1, max_iterations=40, tolerance=0.001 )
animacao_5 = create_line_animation_two_parameters(exemplo5, weights, heights_real, 0.05, "Gradiente Descendente Estocástico")
HTML(animacao_5.to_jshtml())

# Mini-Batch de 2 samples
No Gradiente Descendente Mini-Batch, para dar um único passo, o algoritmo precisa sortear um pequeno subconjunto dos dados, no nosso caso, 2 pontos. Calcular o erro desses dois pontos, somar esses erros, calcular a derivada do somatório dos erros ao quadrado, e então atualizar o $Intercept$ e o $Slope$.

Nesse contexto, a ideia do Mini-Batch de 2 samples é que, ao invés de olhar para todos os dados, como no método padrão, ou para apenas um ponto, como no estocástico, utilizamos dois pontos por iteração para decidir a direção de atualização. Isso torna o método mais estável que o estocástico e mais rápido que o gradiente descendente tradicional.

#Algoritmo
Começamos com valores iniciais, por exemplo, $Intercept = 0$, $Slope = 1$, igual aos outros métodos. Em seguida, ao invés de calcularmos o erro em todos os pontos, sorteiam-se dois pontos de forma aleatória e executa-se o cálculo da derivada com base apenas nesses dois pontos.

Com isso temos que a derivada em relação ao $Intercept$ é:
$$
Gradiente_{intercept} = -2 \sum_{i=1}^{2} \left( y_{real}^{(i)} - y_{pred}^{(i)} \right)
$$
E a atualização do $Intercept$ se dá por:
$$
Intercept_{novo} = Intercept_{velho} - (Learning\ Rate \times Gradiente_{intercept})
$$

Com isso temos que a derivada em relação ao $Slope$ é:
$$
Gradiente_{slope} = -2 \sum_{i=1}^{2} \left( y_{real}^{(i)} - y_{pred}^{(i)} \right)x^{(i)}
$$
E a atualização do $Slope$ se dá por:
$$
Slope_{novo} = Slope_{velho} - (Learning\ Rate \times Gradiente_{slope})
$$

Dessa maneira, após o cálculo do novo $Intercept$ e $Slope$, outro mini-batch contendo dois novos pontos é sorteado e o processo é repetido até alcançarmos uma condição de parada, que pode ser definida por um limite de iterações, por um erro mínimo aceitável ou por um tamanho mínimo do passo.

In [5]:
def mini_batch(learning_rate, start_intercept, x, y, start_slope, max_iterations=40, tolerance=0.001, sample=2):
    # Executa o algoritmo de Gradiente Descendente.

    current_intercept = start_intercept
    current_slope = start_slope
    history = []

    print(f"\n INICIANDO SGD COM LEARNING RATE = {learning_rate}")

    # Adiciona o estado inicial.
    current_ssr = calculate_ssr(current_intercept, current_slope, x, y)
    history.append((current_intercept, current_ssr, 0, 0))

    for i in range(max_iterations):
      # sorteio
      random_index = np.random.choice(len(x), size=sample, replace=False)
      x_i = x[random_index]
      y_i = y[random_index]

      # calcular predict e erro
      predictions = current_intercept + (current_slope * x_i)
      residuals = y_i - predictions

      # gradiente
      d_intercept = -2 * np.sum(residuals)
      d_slope = -2 * np.sum(residuals * x_i)

      # Calcular Passo.
      step_size_intercept = d_intercept * learning_rate
      step_size_slope = d_slope * learning_rate

      # Intercept.
      old_intercept = current_intercept
      new_intercept = old_intercept - step_size_intercept
      current_intercept = new_intercept

      # slope.
      old_slope = current_slope
      new_slope = old_slope - step_size_slope
      current_slope = new_slope

      # Novo Erro.
      current_ssr = calculate_ssr(current_intercept, current_slope, x, y)

      # Salvar Histórico.
      history.append((current_intercept, current_ssr, d_intercept, d_slope))

      print(f"Iteração {i+1}: Old i={old_intercept:.4f} | old s={old_slope:.4f} | d/di={d_intercept:.4f} |d/ds={d_slope:.4f} | Step i={step_size_intercept:.4f} |step s={step_size_slope:.4f} | New i={new_intercept:.4f} | New s={new_slope:.4f}")

      if abs(step_size_intercept) < tolerance and abs(step_size_slope) < tolerance:
          print(f"--> Convergiu na iteração {i+1}!")
          break

    return history

# Exemplo 06: Mini-Batch de 2 Samples
Nesse exemplo, iniciamos com um chute inicial para o  Intercept=0  e  Slope=1

In [ ]:
exemplo6 = mini_batch(0.05, 0, weights, heights_real, 1, max_iterations=1000, tolerance=0.001, sample=2 )
animacao_6 = create_line_animation_two_parameters(exemplo6, weights, heights_real, 0.05, "Mini-Batch de 2 samples")
HTML(animacao_6.to_jshtml())

No $Mini-Batch$ de 2 samples, o uso de dois pontos por iteração proporciona uma estimativa de gradiente mais estável que a do estocástico, suavizando as oscilações nas atualizações do $Intercept$ e do $Slope$. Como consequência, a convergência ocorre de forma mais regular, mantendo boa velocidade de aprendizado sem perder estabilidade.

#Redes Neurais.


Função de ativação.

In [6]:
def activation_function(x):
  return np.log(1+ np.exp(x))

Redes neurais

In [7]:
# valores observados
x_inputs = np.array([0, 0.5, 1])
y_inputs = np.array([0, 1, 0])

def neural_network(learning_rate = .1, max_iterations = 1000, min_step_size = 0.0001, optimize_weights = False):
  # weights
  weights_before_activation = np.array([[3.34], [-3.53]])
  weights_after_activation = np.array([[-1.22], [-2.30]])
  if optimize_weights:
    weights_after_activation = np.array([[0.36], [0.63]])
  # bias
  bias_before_activation = np.array([[-1.43], [0.57]])
  bias_after_activation = np.array([[0.0]])

  return weights_before_activation, weights_after_activation, bias_before_activation, bias_after_activation

def predict_values(x_inputs, weights_before_activation, weights_after_activation, bias_before_activation, bias_after_activation):
  activation_function_inputs = x_inputs * weights_before_activation + bias_before_activation
  activation_function_outputs = activation_function(activation_function_inputs)
  predictions = np.sum(activation_function_outputs * weights_after_activation, axis=0) + bias_after_activation
  return predictions, activation_function_outputs

def evaluate_residual(y_inputs, predctions):
  residuals = y_inputs - predctions
  return residuals

def derivative_ssr_b3(residuals):
  return -2 * np.sum(residuals)

def derivative_ssr_w3(residuals, activation_function_outputs):
  return -2 * np.sum(residuals * activation_function_outputs[0])

def derivative_ssr_w4(residuals, activation_function_outputs):
  return -2 * np.sum(residuals * activation_function_outputs[1])

def training_neural_network(x_inputs, y_inputs, learning_rate=0.1, max_iterations=1000, min_step_size=0.0001, optimize_weights=False):
  w_in, w_out,b_in ,b_3 = neural_network(optimize_weights= optimize_weights)

  print(f"--- INÍCIO DO TREINAMENTO (LR={learning_rate}) ---")
  print(f"Inicial -> b3: {b_3[0][0]:.4f} | w3: {w_out[0][0]:.4f} | w4: {w_out[1][0]:.4f}\n")

  print(f"--- INÍCIO DO TREINAMENTO (Otimizar Pesos: {optimize_weights}) ---")

  for i in range(max_iterations):
    predictions, softplus_outs = predict_values(x_inputs, w_in, w_out, b_in, b_3)

    residuals = evaluate_residual(y_inputs, predictions)

    gradient_b3 = derivative_ssr_b3(residuals)
    step_b3 = gradient_b3 * learning_rate
    print(f"\nIteração {i+1}:")
    old_b3 = b_3[0][0]
    b_3 = b_3 - step_b3
    new_b3 = b_3[0][0]
    print(f"  [b3] Step Size: {step_b3:.6f} | Old: {old_b3:.6f} | New: {new_b3:.6f}")

    if optimize_weights:
      gradient_w3 = derivative_ssr_w3(residuals, softplus_outs[0])
      gradient_w4 = derivative_ssr_w4(residuals, softplus_outs[1])

      step_w3 = gradient_w3 * learning_rate
      step_w4 = gradient_w4 * learning_rate

      old_w3 = w_out[0][0]
      w_out[0][0] = w_out[0][0] - step_w3
      new_w3 = w_out[0][0]
      print(f"  [w3] Step Size: {step_w3:.6f} | Old: {old_w3:.6f} | New: {new_w3:.6f}")

      old_w4 = w_out[1][0]
      w_out[1][0] = w_out[1][0] - step_w4
      new_w4 = w_out[1][0]
      print(f"  [w3] Step Size: {step_w4:.6f} | Old: {old_w4:.6f} | New: {new_w4:.6f}")

      if abs(step_b3) < min_step_size and abs(step_w3) < min_step_size and abs(step_w4) < min_step_size:
        break
    if abs(step_b3) < min_step_size:
      break

  return b_3, w_out









In [8]:
final_b3, final_w_out = training_neural_network(x_inputs, y_inputs, learning_rate=0.1, max_iterations=1000, min_step_size=0.0001, optimize_weights=False)

--- INÍCIO DO TREINAMENTO (LR=0.1) ---
Inicial -> b3: 0.0000 | w3: -1.2200 | w4: -2.3000

--- INÍCIO DO TREINAMENTO (Otimizar Pesos: False) ---

Iteração 1:
  [b3] Step Size: -1.565511 | Old: 0.000000 | New: 1.565511

Iteração 2:
  [b3] Step Size: -0.626204 | Old: 1.565511 | New: 2.191715

Iteração 3:
  [b3] Step Size: -0.250482 | Old: 2.191715 | New: 2.442197

Iteração 4:
  [b3] Step Size: -0.100193 | Old: 2.442197 | New: 2.542390

Iteração 5:
  [b3] Step Size: -0.040077 | Old: 2.542390 | New: 2.582467

Iteração 6:
  [b3] Step Size: -0.016031 | Old: 2.582467 | New: 2.598498

Iteração 7:
  [b3] Step Size: -0.006412 | Old: 2.598498 | New: 2.604910

Iteração 8:
  [b3] Step Size: -0.002565 | Old: 2.604910 | New: 2.607475

Iteração 9:
  [b3] Step Size: -0.001026 | Old: 2.607475 | New: 2.608501

Iteração 10:
  [b3] Step Size: -0.000410 | Old: 2.608501 | New: 2.608911

Iteração 11:
  [b3] Step Size: -0.000164 | Old: 2.608911 | New: 2.609075

Iteração 12:
  [b3] Step Size: -0.000066 | Old: 2.